In [1]:
import itk
import SimpleITK as sitk

In [3]:
import numpy as np

from skimage.filters.rank import entropy
from skimage.morphology import ball
from skimage.measure import shannon_entropy
from scipy.ndimage import generic_filter
import os

In [ ]:
img_dir =r'D:\Data\\IPMNT2_Clustering\\clustering_pre\\nyul_normed'
mask_dir = r'D:\Data\\IPMNT2_Clustering\\cystMask_all_corrected'   
csds = 'D:\Data\IPMNT2_Clustering\cad_cysts'
# save_dir = r'D:\Data\\IPMNT2_Clustering\\cystMask_all_corrected'
for filename in os.listdir(mask_dir):
    if filename.endswith('.nii.gz'):
        if filename.startswith('CAD'):
            try:
                mask_path = os.path.join(mask_dir, filename)

                img_path = os.path.join(img_dir, filename)
                img = sitk.ReadImage(img_path)
                mask = sitk.ReadImage(mask_path)
                mask_array = sitk.GetArrayFromImage(mask)
                # print(img.GetSize(), mask.GetSize())
                print(mask_array.shape)
                mask_array = mask_array[:,::-1,::-1]
                print(mask_array.shape)
                mask = sitk.GetImageFromArray(mask_array)
                mask.SetOrigin(img.GetOrigin())
                mask.SetDirection(img.GetDirection())
                mask.SetSpacing(img.GetSpacing())
                mask.CopyInformation(img)
                print(mask.GetSize())
                sitk.WriteImage(mask, os.path.join(csds, filename))
                # img_arr = sitk.GetArrayFromImage(img)
                # mask_arr = sitk.GetArrayFromImage(mask)
                # img_arr = img_arr * mask_arr
                # img = sitk.GetImageFromArray(img_arr)

                # if img.GetOrigin() != mask.GetOrigin():
                #     print('spacing not equal for ', filename)
                #     mask.SetOrigin(img.GetOrigin())
                #     mask.SetDirection(img.GetDirection())
                #     # sitk.WriteImage(mask, os.path.join(save_dir, filename))
                #     continue

                # print(filename," done")
            except Exception as e:
                print(filename, e)

In [ ]:
import os
import SimpleITK as sitk
import numpy as np
cluster_dir = 'D:\Data\IPMNT2_Clustering\Clustering_mask\\bilateral_n4bias\kmeans Clusters_3'
high_dir = 'D:\Data\IPMNT2_Clustering\Clustering_mask\\bilateral_n4bias\high_intensity_cluster'
for filename in os.listdir(cluster_dir):
    if filename.endswith('.nii.gz'):
        try:
            cluster_path = os.path.join(cluster_dir, filename)
            high_path = os.path.join(high_dir, filename)
            cluster = sitk.ReadImage(cluster_path)
            cluster_array = sitk.GetArrayFromImage(cluster)
            high_array = cluster_array
            # cluster_array = cluster_array * high_array
            # cluster = sitk.GetImageFromArray(cluster_array)
            # cluster.SetOrigin(high.GetOrigin())
            # cluster.SetDirection(high.GetDirection())
            # cluster.SetSpacing(high.GetSpacing())
            # cluster.CopyInformation(high)
            # sitk.WriteImage(cluster, os.path.join(csds, filename))
        except Exception as e:
            print(filename, e)

[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[0. 1.